# Motion Detection (720p) on Arty Z7
This notebook implements a real-time Motion Detection pipeline using Frame Differencing on the PYNQ framework.

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import VideoMode
import cv2
import numpy as np
import threading
import queue
import time
import gc

# Pipeline Configuration
WIDTH, HEIGHT = 1280, 720
PROCESS_SCALE = 0.3  # Process at 384x216 for stability
BUFFER_SIZE = 2
TARGET_FPS = 30
MOTION_THRESHOLD = 25  # Lower = more sensitive

In [ ]:
# Initialize FPGA Overlay and HDMI Output
print("Initializing FPGA...")
base = BaseOverlay("base.bit")
hdmi_out = base.video.hdmi_out

mode = VideoMode(WIDTH, HEIGHT, 24)
hdmi_out.configure(mode)
hdmi_out.start()
print(f"HDMI ready: {WIDTH}x{HEIGHT}")

In [ ]:
# Initialize USB Camera
print("Initializing camera...")
cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
cap.set(cv2.CAP_PROP_FPS, TARGET_FPS)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

print(f"Camera ready: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")

In [ ]:
# Shared State and Queues
frame_queue = queue.Queue(maxsize=BUFFER_SIZE)
result_queue = queue.Queue(maxsize=BUFFER_SIZE)

class StreamState:
    def __init__(self):
        self.running = True
        self.fps_capture = 0.0
        self.fps_process = 0.0
        self.fps_display = 0.0
        self.motion_detected = False
        self.motion_count = 0
        self.dropped_frames = 0
        
state = StreamState()

In [ ]:
# Thread 1: Frame Capture
def capture_thread():
    frame_count = 0
    last_reset = time.perf_counter()
    
    while state.running:
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.001)
            continue
        
        if frame.shape[:2] != (HEIGHT, WIDTH):
            frame = cv2.resize(frame, (WIDTH, HEIGHT))
        
        try:
            frame_queue.put(frame, block=False)
            frame_count += 1
        except queue.Full:
            state.dropped_frames += 1
        
        # Calculate FPS every 30 frames with rolling window
        if frame_count % 30 == 0:
            now = time.perf_counter()
            elapsed = now - last_reset
            state.fps_capture = 30 / elapsed
            last_reset = now

In [ ]:
# Thread 2: Motion Detection (Frame Differencing)
def detection_thread():
    """Lightweight motion detection using frame differencing"""
    prev_gray = None
    process_w = int(WIDTH * PROCESS_SCALE)
    process_h = int(HEIGHT * PROCESS_SCALE)
    
    frame_count = 0
    last_reset = time.perf_counter()
    
    # Morphology kernel for noise reduction
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    while state.running:
        try:
            frame = frame_queue.get(timeout=0.1)
        except queue.Empty:
            continue
        
        # Downscale for processing
        small = cv2.resize(frame, (process_w, process_h))
        gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        
        motion_contours = []
        
        if prev_gray is not None:
            # Frame difference
            diff = cv2.absdiff(prev_gray, gray)
            _, thresh = cv2.threshold(diff, MOTION_THRESHOLD, 255, cv2.THRESH_BINARY)
            
            # Remove noise
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
            thresh = cv2.dilate(thresh, kernel, iterations=2)
            
            # Find contours
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Filter small movements and scale back
            scale_factor = 1.0 / PROCESS_SCALE
            for c in contours:
                if cv2.contourArea(c) > 100:  # Minimum area
                    x, y, w, h = cv2.boundingRect(c)
                    motion_contours.append((
                        int(x * scale_factor),
                        int(y * scale_factor),
                        int(w * scale_factor),
                        int(h * scale_factor)
                    ))
            
            # Update state
            state.motion_detected = len(motion_contours) > 0
            state.motion_count = len(motion_contours)
        
        prev_gray = gray
        
        try:
            result_queue.put((frame, motion_contours), block=False)
            frame_count += 1
        except queue.Full:
            pass
        
        # Calculate FPS every 30 frames with rolling window
        if frame_count % 30 == 0:
            now = time.perf_counter()
            elapsed = now - last_reset
            state.fps_process = 30 / elapsed
            last_reset = now

In [ ]:
# Thread 3: HDMI Display Output
def display_thread():
    prev_time = time.perf_counter()
    fps_smooth = 0.0
    
    hdmi_buffer = hdmi_out.newframe()
    
    while state.running:
        try:
            frame, contours = result_queue.get(timeout=0.1)
        except queue.Empty:
            continue
        
        # Measure actual time between frames
        current_time = time.perf_counter()
        time_diff = current_time - prev_time
        prev_time = current_time
        
        # Draw motion boxes
        for (x, y, w, h) in contours:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Status overlay
        status_color = (0, 255, 0) if state.motion_detected else (100, 100, 100)
        status_text = "MOTION" if state.motion_detected else "NO MOTION"
        cv2.putText(frame, status_text, (20, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, status_color, 2)
        
        # Performance stats
        cv2.putText(frame, f"Capture: {state.fps_capture:.1f} FPS", (20, 95), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Process: {state.fps_process:.1f} FPS", (20, 130), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Display: {fps_smooth:.1f} FPS", (20, 165), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        cv2.putText(frame, f"Motion Regions: {state.motion_count}", (20, 200), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        if state.dropped_frames > 0:
            cv2.putText(frame, f"Dropped: {state.dropped_frames}", (20, 235), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        # Resolution indicator
        cv2.putText(frame, "1280x720", (WIDTH - 180, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)
        
        # Convert and write to HDMI
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hdmi_buffer[:] = frame_rgb
        hdmi_out.writeframe(hdmi_buffer)
        
        # Calculate actual FPS
        if time_diff > 0:
            instant_fps = 1.0 / time_diff
            fps_smooth = 0.9 * fps_smooth + 0.1 * instant_fps
        
        state.fps_display = fps_smooth

In [ ]:
# Main Execution Loop
print("LIGHTWEIGHT MOTION DETECTION PIPELINE (720p)")

try:
    threads = [
        threading.Thread(target=capture_thread, daemon=True),
        threading.Thread(target=detection_thread, daemon=True),
        threading.Thread(target=display_thread, daemon=True)
    ]
    
    for t in threads:
        t.start()
    
    print("Pipeline started")
    
    # Monitor pipeline
    while True:
        time.sleep(5)
        print(f"[Status] FPS: Cap={state.fps_capture:.1f} | Proc={state.fps_process:.1f} | Disp={state.fps_display:.1f} | Drops={state.dropped_frames}")

except KeyboardInterrupt:
    print("\nStopping...")
    state.running = False
    
    for t in threads:
        t.join(timeout=1.0)
    
finally:
    print("Cleanup...")
    cap.release()
    hdmi_out.stop()
    del hdmi_out
    gc.collect()
    print("Resources Released.")